In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%cd DeepLearning_Financial

## EXTERNAL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import yfinance
from pandas import Series
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, PredefinedSplit
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from IPython.display import display
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import time
import os
import random 
from sklearn.datasets import make_regression
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from torch.nn.modules.loss import MSELoss
import tensorflow as tf
from tensorflow import keras
from skorch.dataset import CVSplit
from skorch import callbacks
import pickle
from sklearn.model_selection import train_test_split
from functools import partial
import skorch
import pywt
from sklearn import preprocessing
import joblib


##INTERNAL
from models import Autoencoder, waveletSmooth, SequenceDouble, SequenceDoubleAtt, SequenceAtt
from utils import prepare_data_lstm, ExampleDataset, save_checkpoint, evaluate_lstm, backtest
from testing import *
from sa_tools import *
from test_models import *



#creazione dataset
# indices = ['^DJI', '^GSPC', '^HSI', '^N225', 'ASHR','^NSEI']
indices = ['^GSPC']
feature_sets = ['open', 'ohlcv', 'ext']
start_date="2000-01-01"
end_date="2018-12-31"

# datasets=get_datasets(indices = indices, feature_sets = feature_sets, end_date=end_date, start_date=start_date)
# save(datasets, 'datasets')
datasets = load('^GSCP')
# tss = TimeSeriesSplit(4)
# tss_split = CVSplit(cv=tss, stratified=False, random_state=None)


for index in indices:
    X=datasets[index]["target"]
    
    tss = TimeSeriesSplit(4)
    max_train_size = len(list(tss.split(X))[0][0])
    tss = TimeSeriesSplit(4, max_train_size=max_train_size)
    tss_split = CVSplit(cv=tss, stratified=False, random_state=None)
    
    for i, (train, val) in enumerate(tss.split(X), start=1):
        print("FOLD {}".format(i))
        train_dates = X.index[train]
        val_dates = X.index[val]
        print("Training set da {} a {}".format(train_dates.min(), train_dates.max()))
        print("Validation set da {} a {}".format(val_dates.min(), val_dates.max()))
        print(X.index.min())
        print(X.index.max())
    for i, (train_dates, val_dates) in enumerate(tss.split(datasets[index]["target"]), start=1):
        for model_name in get_model_names():
            
            # get the model
            save_name = "{}_{}_{}_{}.pkl".format(model_name, index, datasets[index]["target"].index[train_dates].min().date(), datasets[index]["target"].index[train_dates].max().date())
            print(save_name)
            model, feature_set = get_model(model_name,  save_name)
                
            #get the dataset
            sa=None
            ld=False
            fset= feature_set
            if(feature_set is 'ext_sa'):
                fset='ext'
                sa = "sa_{}_{}_{}".format(index, datasets[index]["target"][train_dates].index.min().date(), datasets[index]["target"][train_dates].index.max().date())
                sa_fname = sa + ".pkl"
                if(os.path.isfile(sa_fname)):
                    ld=True
            x, y, x_train, x_val, x_scaler, y_train, y_val, y_scaler = get_dataset_train(datasets=datasets, feature_set=fset, train_dates=train_dates, val_dates=val_dates, index=index, sa=sa, ld=ld)
            
            #prepare dataset for training
            n_days=5

            x_batch = days_group(x, n_days=n_days)
            y_batch = y[n_days:]
            serie = x_batch
            train_dates_s = train_dates[n_days:].copy()
            val_dates_s = val_dates.copy()

#             opn_test = y_scaler.transform(datasets[index]["original"]["Open"].iloc[val_dates].to_numpy().reshape(-1,1))
            opn_test = datasets[index]["original"]["Open"].iloc[val_dates].to_numpy()
            roi_test = partial(roi_train,opn=opn_test)
            
            model, feature_set = get_model(model_name,  save_name, roi_test=roi_test)
            
            model.y_scaler = y_scaler
            model.x_scaler = x_scaler
            
            splitted = np.split(serie, [len(train_dates_s)])
            l1, l2 = len(splitted[0]), len(splitted[1])
            train_s = PredefinedSplit(np.concatenate((np.ones(l1)*-1,np.ones(l2))))
            train_split =  CVSplit(cv=train_s, stratified=False, random_state=None)  
            train_dates_s, val_dates_s = list(train_s.split(serie))[0]
            
            model.train_split = train_split
            model.fit(x_batch,y_batch)
    

/data/home/dsvm_server_admin/notebooks/fastai/tesi/DeepLearning_Financial
FOLD 1
Training set da 2000-09-19 00:00:00 a 2004-05-18 00:00:00
Validation set da 2004-05-19 00:00:00 a 2008-01-12 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
FOLD 2
Training set da 2004-05-15 00:00:00 a 2008-01-12 00:00:00
Validation set da 2008-01-13 00:00:00 a 2011-09-07 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
FOLD 3
Training set da 2008-01-09 00:00:00 a 2011-09-07 00:00:00
Validation set da 2011-09-08 00:00:00 a 2015-05-03 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
FOLD 4
Training set da 2011-09-04 00:00:00 a 2015-05-03 00:00:00
Validation set da 2015-05-04 00:00:00 a 2018-12-27 00:00:00
2000-09-19 00:00:00
2018-12-27 00:00:00
lstm_sa_^GSPC_2000-09-19_2004-05-18.pkl
  epoch    neg_mean_absolute_error       r2    roi_train    train_loss    valid_loss    cp     dur
-------  -------------------------  -------  -----------  ------------  ------------  ----  ------
      1                    -

/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


      2                    -1.4280  -2.7062     -72.7016        1.0166        2.7882        2.2954
      3                    -1.4287  -2.7087     -72.7016        1.0109        2.7900        1.9940
      4                    -1.4285  -2.7079     -72.7016        1.0065        2.7895        1.3343
      5                    -1.4279  -2.7059     -72.7016        1.0034        2.7879        1.6327
      6                    -1.4273  -2.7036     -72.7016        1.0011        2.7862        1.5431
      7                    -1.4268  -2.7015     -72.7016        0.9994        2.7846        1.9824
      8                    -1.4263  -2.6994     -72.7016        0.9980        2.7831        1.5867
      9                    -1.4258  -2.6978     -72.7016        0.9970        2.7818        1.7226
     10                    -1.4254  -2.6962     -72.7016        0.9961        2.7806        1.8483
     11                    -1.4249  -2.6944     -72.7016        0.9953        2.7793        1.8054
     12   

/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


     18                    -1.4223  -2.6843     -72.5763        0.9916        2.7717     +  2.5447


...

    317                    -0.8820  -0.4626     -28.3846        0.0664        1.1003     +  2.3029


/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


    318                    -0.8794  -0.4555     -28.3846        0.0655        1.0949     +  1.5593


/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


    319                    -0.8746  -0.4425     -27.5251        0.0644        1.0852     +  1.8258
